# Perform Node Classification on the WACV Dataset

**Overview of the dataset**
- The new dataset is in `./GraphPCB/Graph-W/graphs/` and `/home/lantian/GraphPCB/Graph-F/graphs/`
- The nodes have 4 types/categories:
    - 0-IC  the target
    - 1-DT (hard to distinguish from 0-IC)
    - 2-Diode (also hard to tell from 0-IC)
    - 3-Others (everything else, should be easier to distinguish)
- **Note that not all categories are presented in all the graphs (which means, there are some graphs that has less than 4 types of nodes)**

**Graph-WACV**
- The train/test sets contain 37/10 graphs each.
- The stats of the graphs are like this:
    - avg num of nodes in each graph are mostly around 50-600
    - avg num of node degrees are between 5.7-5.9
    - the diameter of the graph are a normal distribution centered at ~14

**Plans**
- Try some node classification that presents in this repo

In [1]:
import os
import json
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from glob import glob
import shutil
import random
from utils import *
from logger import PCB_Logger

from collections import defaultdict
from acmgnn.models import GCN

from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

if torch.cuda.is_available():
    from torch_geometric.utils import (
        to_dense_adj,
        contains_self_loops,
        remove_self_loops,
    )

# Dataset Loader

- Load GraphPCB graphs as DGLGraphs

In [7]:
# Set the home directory for the dataset
home_dir = os.path.expanduser("~")
dataset_dir = os.path.join(home_dir, "GraphPCB_Analysis/GraphPCB")
print("Dataset directory:", dataset_dir)

Dataset directory: /home/lantian/GraphPCB_Analysis/GraphPCB


In [11]:
def get_data_loader(config):
    train_dir = os.path.join(config['dataset_dir'], f"Graph-{config['dataset'][0].upper()}/graphs", "train")
    test_dir = os.path.join(config['dataset_dir'], f"Graph-{config['dataset'][0].upper()}/graphs", "test")
    train_graphs = sorted(glob(f"{train_dir}/*.pt"))
    test_graphs = sorted(glob(f"{test_dir}/*.pt"))
    return train_graphs, test_graphs

In [4]:
def load_our_data(graph_path, device):
    """
    Load user's .pt graph file in PyG Data format and return
    sparse normalized adjacency matrices (low/high frequency),
    features, and labels.
    """
    data = torch.load(graph_path, map_location=device)

    x = data.x.to(device)  # Node features
    y = data.y.to(device)  # Node labels
    edge_index = data.edge_index.to(device)

    num_nodes = x.size(0)

    # Convert edge_index to dense adjacency
    adj_dense = to_dense_adj(edge_index, max_num_nodes=num_nodes)[0]
    adj_dense[adj_dense != 0] = 1
    adj_dense.fill_diagonal_(0)

    # Compute normalized low- and high-pass filters
    adj_low = adj_dense + torch.eye(num_nodes, device=device)
    deg = torch.sum(adj_low, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    norm = deg_inv_sqrt.unsqueeze(1) * adj_low * deg_inv_sqrt.unsqueeze(0)
    adj_low = norm.to_sparse()
    adj_high = (torch.eye(num_nodes, device=device) - norm).to_sparse()

    return adj_low, adj_high, x, y

# Inference

In [5]:
def inference(model, test_graphs):
    model.eval()
    all_preds, all_labels = [], []
    predictions = []

    for graph_path in test_graphs:
        # get device fron model
        device = next(model.parameters()).device
        graph_file_name = os.path.basename(graph_path)
        adj_low, adj_high, features, labels = load_our_data(graph_path, device)
        features, labels = features.to(device), labels.to(device)
        adj_low, adj_high = adj_low.to(device), adj_high.to(device)

        with torch.no_grad():
            output = model(features, adj_low, adj_high)
            preds = output.max(1)[1]
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

        predictions.append({
            "graph_id": graph_file_name,
            "labels": preds.tolist()
        })

    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    return all_preds.numpy(), all_labels.numpy(), predictions


# Training Function
- for every 10 epoch, save the model and evaluate on test set
- save the training log into a .txt
- save the last checkpoint
- save the predictions into a .json

In [6]:
def train_step(model, train_graphs, optimizer, scheduler, device):
    total_loss = 0
    model.train()

    optimizer.zero_grad()
    
    for i, graph_path in enumerate(train_graphs):
        adj_low, adj_high, features, labels = load_our_data(graph_path, device)

        features, labels = features.to(device), labels.to(device)
        adj_low, adj_high = adj_low.to(device), adj_high.to(device)

        logits = model(features, adj_low, adj_high)
        loss = compute_loss(logits, labels)

        loss.backward()
        total_loss += loss.item()

        optimizer.step()
        
    scheduler.step()

    avg_loss = total_loss / len(train_graphs)
    return avg_loss


In [8]:
# the main function to train the model
def train_model(config):
    """
    Generic function to train different GNN models.
    """
    set_seed(42)
    train_graphs, test_graphs = get_data_loader(config)

    logger = PCB_Logger(home_dir='/home/lantian/', config=config)

    torch.cuda.empty_cache()

    # Load first graph to init model
    adj_low, adj_high, features, labels = load_our_data(train_graphs[0], config["device"])
    model = GCN(
        nfeat=features.shape[1],
        nhid=config["hidden_dim"],
        nclass=labels.max().item() + 1,
        dropout=config["dropout"],
        model_type=config["model"],
    ).to(config["device"])

    
    # ✅ Move model to device
    model = model.to(config["device"])
    
    # ✅ Define optimizer & scheduler
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config["scheduler"]["step_size"], gamma=config["scheduler"]["gamma"])

    # ✅ Training Loop
    for epoch in range(config["num_epochs"]):
        avg_loss = train_step(model, train_graphs, optimizer, scheduler, config["device"])
        logger.log(f"Epoch {epoch + 1:03d}, Loss: {avg_loss:.10f}")

        for name, param in model.named_parameters():
            if not param.requires_grad:
                print(f"Parameter with requires_grad=False: {name}")

        # ✅ Evaluate every 10 epochs OR at the last epoch
        if (epoch + 1) % 10 == 0 or (epoch + 1 == config["num_epochs"]):
            all_preds, all_labels, predictions = inference(model, test_graphs)
            metrics = compute_metrics(all_preds, all_labels)
            logger.update_metrics(metrics, predictions)
    
    logger.finish_run()
    # save only the final model
    checkpoint_path = os.path.join(logger.checkpoint_dir, f"model_epoch_{epoch + 1}_{avg_loss:.4f}.pth")
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'metrics': metrics,
    }, checkpoint_path)
    logger.log(f"✅ Model checkpoint saved to {checkpoint_path}")

    return model, metrics, logger.checkpoint_dir

# Config and Args

In [12]:
# Basic configuration 
config = {
    "experiment_name": "",
    "dataset_dir": dataset_dir,
    "home_dir": home_dir,
    "dataset": "fpic",
    "device": "cuda:0",

    # model architecture
    "model": "acmgcn",
    "input_dim": 1024,
    "hidden_dim": 256,
    "output_dim": 4,

    # regularization
    "dropout": 0.3,
    "weight_decay": 1e-4,
    "scheduler": {"type": "StepLR", "step_size": 20, "gamma": 0.5},

    # training parameters
    "learning_rate": 1e-4,
    "num_epochs": 200,  
}

In [13]:
run_num = 0

config.update({
    "dataset": "fpic",

    # model architecture
    "model": "acmgcn",
    "hidden_dim": 256,
    # regularization
    "dropout": 0.5,
    "weight_decay": 1e-2,

    # training parameters
    "learning_rate": 1e-4,
    "num_epochs": 200,
})

config["experiment_name"] = f"ACMGNN-{config['model']}_h{config['hidden_dim']}_{run_num}-NLL"

# Set seed before training
set_seed(42)

model, metrics, checkpoint_dir = train_model(config)

Checkpoint directory: /home/lantian/GraphPCB_Analysis/Graph-F-trained/ACMGNN-acmgcn_h256_0-NLL
Experiment Configuration:
experiment_name: ACMGNN-acmgcn_h256_0-NLL
dataset_dir: /home/lantian/GraphPCB_Analysis/GraphPCB
home_dir: /home/lantian
dataset: fpic
device: cuda:0
model: acmgcn
input_dim: 1024
hidden_dim: 256
output_dim: 4
dropout: 0.5
weight_decay: 0.01
scheduler: {'type': 'StepLR', 'step_size': 20, 'gamma': 0.5}
learning_rate: 0.0001
num_epochs: 200
Using device: cuda:0
Loading dataset: fpic
Results will be saved to /home/lantian/GraphPCB_Analysis/Graph-F-trained/ACMGNN-acmgcn_h256_0-NLL.
Epoch 001, Loss: 1.2414564313
Epoch 002, Loss: 1.0352419660
Epoch 003, Loss: 0.8434374422
Epoch 004, Loss: 0.7830112258
Epoch 005, Loss: 0.7294144131
Epoch 006, Loss: 0.6955002044
Epoch 007, Loss: 0.7207780249
Epoch 008, Loss: 0.6694758042
Epoch 009, Loss: 0.6743271260
Epoch 010, Loss: 0.6579381474
  F1-Score (macro): 0.5298613004
  Weighted F1: 0.9224893322
  Subset F1-Score (3-class): 0.65226

FileNotFoundError: [Errno 2] No such file or directory: '/home/lantian//GraphPCB_Analysis/Graph-F-trained/MLP/predictions_final.json'

# Run Multiple
- Run multiple experiments for all the settings
    - `model` = `acmgcn` or `acmsgc`
    - `hidden_dim` from 64 to 1024

In [12]:
# Define the settings to iterate over
dataset_variants = ['FPIC', 'WACV']
model_variants = ['acmcgn', 'acmsgc']
hidden_dims = [64, 128, 256]

# Iterate over the settings
for dataset in dataset_variants:
    for model_variant in model_variants:
        for hidden_dim in hidden_dims:
            # Update the config with the current settings
            config.update({
                "dataset": dataset,
                "model": model_variant,
                "hidden_dim": hidden_dim,
                "experiment_name": f"{dataset}_{model_variant}_h{hidden_dim}",
            })

            # Train the model with the updated config
            model, metrics, checkpoint_dir = train_model(config)


Checkpoint directory: /home/lantian/PCB_Analysis/FPIC-trained/FPIC_acmcgn_h64
Experiment Configuration:
experiment_name: FPIC_acmcgn_h64
dataset_dir: /home/lantian/GraphPCB/
home_dir: /home/lantian/
dataset: FPIC
device: cuda:0
model: acmcgn
input_dim: 1024
hidden_dim: 64
output_dim: 4
dropout: 0.5
weight_decay: 0.01
scheduler: {'type': 'StepLR', 'step_size': 20, 'gamma': 0.5}
learning_rate: 0.0001
num_epochs: 200
Using device: cuda:0
Loading dataset: FPIC
Results will be saved to /home/lantian/PCB_Analysis/FPIC-trained/FPIC_acmcgn_h64.


ValueError: optimizer got an empty parameter list